# MESSAGEix Canada

Once we have gone through the `MESSAGE Introduction` notebook, we will now use a real case study for Canada. The data required for the model is compiled in the `data_CA.xlsx` file and we use python interface to solve it.



## Initialize scenario

In [1]:
import message_ix
import ixmp as ix
from ixmp import Scenario
from message_ix import make_df
#from functions.format_demand import format_demand
import pandas as pd
from pathlib import Path
from message_ix.reporting import Reporter
from update_units import unit_correction
import numpy as np


# Stand-alone model
modelName = 'MESSAGEix_CA'      # A MESSAGE model name
scenarioName = 'baseline'       # A MESSAGE model scenario name
ssp = 'SSP1'

mp = ix.Platform()

# Creating a new MESSAGE stand-alone model
msg = message_ix.Scenario(mp, modelName, scenarioName,
                            version='new', annotation='')

<IPython.core.display.Javascript object>

In [2]:
# Path to data & output folder
DATA = Path().cwd()/ 'data'
OUTPUT = Path().cwd()/ 'output'

### Read model data

In [ ]:
# Read in parameter data from previous MESSAGE run
msg.read_excel(DATA / 'data_CA.xlsx', 
               add_units=True, 
               commit_steps=True,
               init_items = True)

## Solve model

In [ ]:
msg.model
msg.version


# Exporting the built model (Scenario) to GAMS with an optional case name
caseName = msg.model + '__' + msg.scenario + '__v' + str(msg.version)

# Solving the model
msg.solve(model='MESSAGE', case=caseName)

## Fixing Units for parameters

In [ ]:
#%% Generating 

unit_correct = True  # unit s must be corrected to run this script
from_excel = False
min_year = 2015
max_year = 2060

if unit_correct:
    msg = msg.clone(keep_solution=False)
    unit_correction(mp, msg)
    caseName = msg.model + '__' + msg.scenario + '__v' + str(msg.version)
    # Solving the model
    msg.solve(model='MESSAGE', case=caseName, solve_options={'lpmethod': '4'})


## Generating Report

In [ ]:

rep = Reporter.from_scenario(msg)
report = rep.get('message::default') # in fact, only in = input x ACT, out = output x ACT is needed


## Adjusting Variable names with appropriate nomenclatures

In [ ]:
report_df = report.timeseries()
report_df.reset_index(inplace=True)
report_df.columns = report_df.columns.astype(str)
report_df.columns = report_df.columns.str.title()

# Plotted years
years = msg.set('year').tolist()
years = [str(x) for x in years if x >= min_year and x <= max_year]

# Create data for Sankey diagrams and plot it using plotly
report_df.fillna(0, inplace = True)

# Convert from GWa to TWh (*8.76)
c = report_df.select_dtypes(include=[np.number]) * 8.76
report_df[c.columns] = c

# Drop not necessary years
report_df.columns = report_df.columns.astype(str)
report_df = report_df.drop(report_df.columns.to_series()['2020': str(min_year)],
                        axis=1)

# Identify input and output variables and its flows (provided within the Reporter: in = input x ACT, out = output x ACT)
in_flows = report_df[report_df['Variable'].str.startswith("in|")]
out_flows = report_df[report_df['Variable'].str.startswith("out|")]
all_flows = pd.concat([out_flows, in_flows])

report_df["Variable"] = report_df["Variable"].str.replace('emis', 'Emissions', regex = False)
#report_df["Variable"] = report_df["Variable"].str.replace('in|final|', 'Final Energy|', regex = False)
report_df["Variable"] = report_df["Variable"].str.replace('out|final|', 'Final Energy|', regex = False)
#report_df["Variable"] = report_df["Variable"].str.replace('in|primary|', 'Primary Energy|', regex = False)
report_df["Variable"] = report_df["Variable"].str.replace('out|primary|', 'Primary Energy|', regex = False)
report_df["Variable"] = report_df["Variable"].str.replace('in|renewable|', 'Primary Energy|renewable|', regex = False)
report_df["Variable"] = report_df["Variable"].str.replace('out|renewable|', 'Primary Energy|renewable|', regex = False)
report_df["Variable"] = report_df["Variable"].str.replace('total om cost|', 'Total Costs|', regex = False)
report_df["Variable"] = report_df["Variable"].str.replace('out|useful|', 'Useful Energy|', regex = False)
report_df["Variable"] = report_df["Variable"].str.replace('in|useful|', 'Useful Energy|', regex = False)


In [ ]:
report_df.to_csv(OUTPUT /'MESSAGE_CANADA_output.csv', index = False)

## Assignments

Hint: Follow the MESSAGEix tutorials [tutorials ](https://github.com/iiasa/message_ix/tree/main/tutorial/westeros)

### 1. Plot the model results following the tutorial
    a) What plots do you think could be useful
    b) What do you understand from the current results 
    c) Does the results look pluasible, if not, what are the differences. (Use historical data to match the future results from the model)
    
### 2. Brainstrom on the possible scenarios of Candian energy transition 
    a) How can we translate the scenario from the baseline. Show an example. 
    b) How can we implement carbon tax in the canadian model
    Hint: Look through Westeros_emission_taxes tutorial and translate into the Canadian landscape

### 3. Understanding economic costs 
    a) What do you understand from the investment costs, fixed costs, variable costs in MESSAGE landscape. 
    b) How do you think the economic costs can help us understand the energy transition process. Give examples from the model. 



In [ ]:
mp.close_db()